<a href="https://colab.research.google.com/github/amokhtare-ivsonance/AcousTools/blob/main/Rake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Install necessary library
!pip install gdspy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.9/157.9 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gdspy: filename=gdspy-1.6.13-cp312-cp312-linux_x86_64.whl size=595857 sha256=08bdf0edf946a02b32ce500f482633f42634467ee0e163d6f4a87f3da70f3077
  Stored in directory: /root/.cache/pip/wheels/02/05/a3/c4b581f8330cedff4d7f4aa7134ae298e88232ccce6a2d4859
Successfully built gdspy


In [ ]:
import numpy as np
import sympy as sp
import gdspy
import os

# Clear library to prevent naming collisions on re-runs
gdspy.current_library = gdspy.GdsLibrary()

def generate_designs():
    # Constants
    cglass = 3280.0  # m/s speed in glass
    cWater = 1523.0  # m/s speed in water at 37 C
    m = 1
    l = 1
    GapJumps = 2

    # Define symbolic variables
    theta, thetac, thick, z, k_sym = sp.symbols('theta thetac thick z k')

    # Designs loop: 6 to 8 with step 2
    for designs in range(6, 9, 2):
        print(f"Processing Rake design s={designs} with fixed electrodes...")

        s = designs

        # Frequency setup
        freqhl1 = np.arange(44.0e6, 43.99e6, -0.01e6)
        if len(freqhl1) == 0: freqhl1 = np.array([44.0e6])
        freqhl = freqhl1
        nc = 1
        freq = np.tile(freqhl, nc)

        # Calculate lambda and k
        lam = cglass / freq
        kl = 2 * np.pi / lam
        kl_val = kl[0]

        zfix = 1100e-6 + 0e-6
        zfixbar = kl * zfix

        Fn = 12
        FFn = 1
        DegreeAngle = 0 * np.pi / 4

        strv = f"RakeTrap-{l}{s}-(44)-Cont{Fn}"

        # Initial angles
        val_for_floor = (zfixbar[0]/m + np.pi) / (2*np.pi)
        base_angle = 2 * np.pi * np.floor(val_for_floor) + 2 * np.pi

        iniang = (base_angle + 1 * DegreeAngle) * np.ones_like(zfixbar)
        iniangj = (base_angle) * np.ones_like(zfixbar)

        discth = 100
        angth = 2 * np.pi / discth

        # ---------------------------------------------------------
        # SYMBOLIC MATH PREPARATION
        # ---------------------------------------------------------
        expr_rho1 = (1/kl_val) * sp.sqrt((m*(theta) + l*thetac)**2 - z**2)
        expr_rho2 = (1/kl_val) * sp.sqrt((m*(theta - sp.pi) + l*thetac)**2 - z**2)

        expr_X1 = expr_rho2 * sp.cos(theta)
        expr_Y1 = expr_rho2 * sp.sin(theta)
        expr_X2 = expr_rho1 * sp.cos(theta)
        expr_Y2 = expr_rho1 * sp.sin(theta)

        dX1_dth = sp.diff(expr_X1, theta)
        dY1_dth = sp.diff(expr_Y1, theta)
        norm1 = sp.sqrt(dX1_dth**2 + dY1_dth**2)
        expr_Nx1 = -dY1_dth / norm1
        expr_Ny1 = dX1_dth / norm1

        dX2_dth = sp.diff(expr_X2, theta)
        dY2_dth = sp.diff(expr_Y2, theta)
        norm2 = sp.sqrt(dX2_dth**2 + dY2_dth**2)
        expr_Nx2 = -dY2_dth / norm2
        expr_Ny2 = dX2_dth / norm2

        expr_x1lower = expr_X1 + expr_Nx1 * thick / 4
        expr_y1lower = expr_Y1 + expr_Ny1 * thick / 4
        expr_x1upper = expr_X1 - expr_Nx1 * thick / 4
        expr_y1upper = expr_Y1 - expr_Ny1 * thick / 4

        expr_x2lower = expr_X2 + expr_Nx2 * thick / 4
        expr_y2lower = expr_Y2 + expr_Ny2 * thick / 4
        expr_x2upper = expr_X2 - expr_Nx2 * thick / 4
        expr_y2upper = expr_Y2 - expr_Ny2 * thick / 4

        # Lambdify
        func_rho1 = sp.lambdify((theta, thetac, z), expr_rho1, "numpy")
        func_rho2 = sp.lambdify((theta, thetac, z), expr_rho2, "numpy")

        func_x1L = sp.lambdify((theta, thetac, thick, z), expr_x1lower, "numpy")
        func_y1L = sp.lambdify((theta, thetac, thick, z), expr_y1lower, "numpy")
        func_x1U = sp.lambdify((theta, thetac, thick, z), expr_x1upper, "numpy")
        func_y1U = sp.lambdify((theta, thetac, thick, z), expr_y1upper, "numpy")

        func_x2L = sp.lambdify((theta, thetac, thick, z), expr_x2lower, "numpy")
        func_y2L = sp.lambdify((theta, thetac, thick, z), expr_y2lower, "numpy")
        func_x2U = sp.lambdify((theta, thetac, thick, z), expr_x2upper, "numpy")
        func_y2U = sp.lambdify((theta, thetac, thick, z), expr_y2upper, "numpy")

        # ---------------------------------------------------------
        # GEOMETRY GENERATION
        # ---------------------------------------------------------

        # Collect polygons
        spiral_inner_polys = []
        spiral_outer_polys = []

        saveLowP0 = {}
        saveupP0 = {}
        saveLowPP = {}
        saveupPP = {}

        ix = 0
        h = np.arange(0, s * Fn + 1)

        top_cell_name = f"TopSpiral_{designs}"
        main_cell = gdspy.Cell(top_cell_name)

        # Variables to store the 'cut' shapes
        cxy1, cxy2, cxy1o, cxy2o = None, None, None, None
        jxy1, jxy2 = None, None

        # --- CAPTURE VARIABLES FOR ELECTRODES ---
        final_jxyup = None   # Right Tip
        final_jxybup = None  # Left Tip

        for ii in range(FFn, int(s * np.floor(Fn)) + 1):

            X1L_list, Y1L_list = [], []
            X1U_list, Y1U_list = [], []
            X2L_list, Y2L_list = [], []
            X2U_list, Y2U_list = [], []

            for oo in range(len(freq)):
                start_angle = iniang[0] + (2 * np.pi / s) * h[ii-1]
                end_angle = iniang[0] + (2 * np.pi / s) * h[ii] - GapJumps * angth
                Angle = np.arange(start_angle, end_angle + angth/100, angth)

                Anglec1 = np.linspace(0, 2 * np.pi, len(Angle))
                Anglec2 = np.linspace(2 * np.pi, 0, len(Angle))

                if (ii % s > s / 2) or (ii % s == 0):
                    Anglec = Anglec2
                else:
                    Anglec = Anglec1

                r1_vals = func_rho1(Angle, Anglec, zfixbar[oo])
                r2_vals = func_rho2(Angle, Anglec, zfixbar[oo])
                thickness = np.abs(r1_vals - r2_vals)

                x1l = func_x1L(Angle, Anglec, thickness, zfixbar[oo])
                y1l = func_y1L(Angle, Anglec, thickness, zfixbar[oo])
                x1u = func_x1U(Angle, Anglec, thickness, zfixbar[oo])
                y1u = func_y1U(Angle, Anglec, thickness, zfixbar[oo])

                X1L_list.extend(x1l)
                Y1L_list.extend(y1l)
                X1U_list.extend(x1u)
                Y1U_list.extend(y1u)

                Angle2 = Angle
                x2l = func_x2L(Angle2, Anglec, thickness, zfixbar[oo])
                y2l = func_y2L(Angle2, Anglec, thickness, zfixbar[oo])
                x2u = func_x2U(Angle2, Anglec, thickness, zfixbar[oo])
                y2u = func_y2U(Angle2, Anglec, thickness, zfixbar[oo])

                X2L_list.extend(x2l)
                Y2L_list.extend(y2l)
                X2U_list.extend(x2u)
                Y2U_list.extend(y2u)

            X1L, Y1L = np.array(X1L_list), np.array(Y1L_list)
            X1U, Y1U = np.array(X1U_list), np.array(Y1U_list)
            X2L, Y2L = np.array(X2L_list), np.array(Y2L_list)
            X2U, Y2U = np.array(X2U_list), np.array(Y2U_list)

            scale = 1e6
            xy1lowP0 = np.column_stack((X1L, Y1L)) * scale
            xy1upP0 = np.column_stack((X1U, Y1U)) * scale
            xy2lowPP = np.column_stack((X2L, Y2L)) * scale
            xy2upPP = np.column_stack((X2U, Y2U)) * scale

            saveLowP0[ii] = xy1lowP0[-1, :]
            saveupP0[ii] = xy1upP0[-1, :]
            saveLowPP[ii] = xy2lowPP[-1, :]
            saveupPP[ii] = xy2upPP[-1, :]

            if ii > 1:
                xy1lowP0 = np.vstack((saveLowP0[ii-1], xy1lowP0))
                xy1upP0 = np.vstack((saveupP0[ii-1], xy1upP0))
                xy2lowPP = np.vstack((saveLowPP[ii-1], xy2lowPP))
                xy2upPP = np.vstack((saveupPP[ii-1], xy2upPP))

            xy1upP0inverse = xy1upP0[::-1]
            xy2upPPinverse = xy2upPP[::-1]

            xy1 = np.vstack((xy1lowP0, xy1upP0inverse))
            xy2 = np.vstack((xy2lowPP, xy2upPPinverse))

            # Store polygons for boolean ops later
            poly1 = gdspy.Polygon(xy1, layer=1)
            poly2 = gdspy.Polygon(xy2, layer=1)
            spiral_inner_polys.append(poly1)
            spiral_outer_polys.append(poly2)

            # ---------------------------
            # Electrode / Jumper Calculation
            # ---------------------------
            gapAngle = np.pi / 24
            jthetaT_val = iniang[0]
            jAnglec_val = Anglec[0]

            jt_r1 = func_rho1(jthetaT_val, jAnglec_val, zfixbar[0])
            jt_r2 = func_rho2(jthetaT_val, jAnglec_val, zfixbar[0])
            jthickT_val = np.abs(jt_r1 - jt_r2)

            def get_pt(ang, ac, thk, zf, func_x, func_y):
                     return np.array([float(func_x(ang, ac, thk, zf)), float(func_y(ang, ac, thk, zf))])

            if ix == 0:
                # Jumper Logic
                p1 = get_pt(iniang[0]-gapAngle/3, Anglec[0], jthickT_val, zfixbar[0], func_x1L, func_y1L)
                p2 = get_pt(iniang[0]+gapAngle/3, Anglec[0], jthickT_val, zfixbar[0], func_x1L, func_y1L)
                jxylow = np.vstack((p1, p2)) * 1e6

                p3 = get_pt(iniang[0]+np.pi-gapAngle/3, Anglec[0], jthickT_val, zfixbar[0], func_x1L, func_y1L)
                p4 = get_pt(iniang[0]+np.pi+gapAngle/3, Anglec[0], jthickT_val, zfixbar[0], func_x1L, func_y1L)
                jxyblow = np.vstack((p3, p4)) * 1e6

                # Central Electrode Cuts
                cp1 = get_pt(iniang[0], Anglec[0], jthickT_val, zfixbar[0], func_x1L, func_y1L)
                cp2 = get_pt(iniang[0]+gapAngle/2, Anglec[0], jthickT_val, zfixbar[0], func_x1L, func_y1L)
                cxylow = np.vstack((cp1, cp2)) * 1e6

                cp3 = get_pt(iniang[0]+np.pi, Anglec[0], jthickT_val, zfixbar[0], func_x1L, func_y1L)
                cp4 = get_pt(iniang[0]+np.pi+gapAngle/2, Anglec[0], jthickT_val, zfixbar[0], func_x1L, func_y1L)
                cxyblow = np.vstack((cp3, cp4)) * 1e6

                # Outer Cuts
                cpo1 = get_pt(iniang[0]-gapAngle/2, Anglec[0], jthickT_val, zfixbar[0], func_x1L, func_y1L)
                cpo2 = get_pt(iniang[0], Anglec[0], jthickT_val, zfixbar[0], func_x1L, func_y1L)
                cxylowo = np.vstack((cpo1, cpo2)) * 1e6

                cpo3 = get_pt(iniang[0]+np.pi-gapAngle/2, Anglec[0], jthickT_val, zfixbar[0], func_x1L, func_y1L)
                cpo4 = get_pt(iniang[0]+np.pi, Anglec[0], jthickT_val, zfixbar[0], func_x1L, func_y1L)
                cxyblowo = np.vstack((cpo3, cpo4)) * 1e6

            elif ii == int(np.floor(Fn)):
                ang_end = iniang[0] + 2*np.pi*Fn
                ang_c_end = Anglec[-1]

                # Jumper (Right)
                p1 = get_pt(ang_end+gapAngle/3, ang_c_end, jthickT_val, zfixbar[0], func_x1U, func_y1U)
                p2 = get_pt(ang_end-gapAngle/3, ang_c_end, jthickT_val, zfixbar[0], func_x1U, func_y1U)
                jxyup = np.vstack((p1, p2)) * 1e6 * np.array([1.2, 1.3])

                final_jxyup = jxyup # CAPTURE RIGHT TIP

                # Jumper (Left)
                p3 = get_pt(ang_end+np.pi+gapAngle/3, ang_c_end, jthickT_val, zfixbar[0], func_x1U, func_y1U)
                p4 = get_pt(ang_end+np.pi-gapAngle/3, ang_c_end, jthickT_val, zfixbar[0], func_x1U, func_y1U)
                jxybup = np.vstack((p3, p4)) * 1e6 * np.array([1.2, 1.0])

                final_jxybup = jxybup # CAPTURE LEFT TIP

                # Central Cuts
                cp1 = get_pt(ang_end+gapAngle/2, ang_c_end, jthickT_val, zfixbar[0], func_x1U, func_y1U)
                cp2 = get_pt(ang_end, ang_c_end, jthickT_val, zfixbar[0], func_x1U, func_y1U)
                cxyup = np.vstack((cp1, cp2)) * 1.5e6 * np.array([1.0, 1.0])

                cp3 = get_pt(ang_end+np.pi+gapAngle/2, ang_c_end, jthickT_val, zfixbar[0], func_x1U, func_y1U)
                cp4 = get_pt(ang_end+np.pi, ang_c_end, jthickT_val, zfixbar[0], func_x1U, func_y1U)
                cxybup = np.vstack((cp3, cp4)) * 1.0e6 * np.array([1.5, 1.0])

                # Outer Cuts
                cpo1 = get_pt(ang_end, ang_c_end, jthickT_val, zfixbar[0], func_x1U, func_y1U)
                cpo2 = get_pt(ang_end-gapAngle/2, ang_c_end, jthickT_val, zfixbar[0], func_x1U, func_y1U)
                cxyupo = np.vstack((cpo1, cpo2)) * 1.5e6

                cpo3 = get_pt(ang_end+np.pi, ang_c_end, jthickT_val, zfixbar[0], func_x1U, func_y1U)
                cpo4 = get_pt(ang_end+np.pi-gapAngle/2, ang_c_end, jthickT_val, zfixbar[0], func_x1U, func_y1U)
                cxybupo = np.vstack((cpo3, cpo4)) * 1.0e6 * np.array([1.5, 1.0])

            ix += 1

        # ------------------------------------------------
        # BOOLEAN OPERATIONS & ASSEMBLY
        # ------------------------------------------------

        # 1. Define the Cut Shapes
        cxy1 = np.vstack((cxylow, cxyup))
        cxy2 = np.vstack((cxyblow, cxybup))
        cxy1o = np.vstack((cxylowo, cxyupo))
        cxy2o = np.vstack((cxyblowo, cxybupo))

        cElectrodeTop = gdspy.Polygon(cxy1, layer=3)
        cElectrodeBot = gdspy.Polygon(cxy2, layer=3)
        cElectrodeTopo = gdspy.Polygon(cxy1o, layer=3)
        cElectrodeBoto = gdspy.Polygon(cxy2o, layer=3)

        # 2. Combine Spiral Parts
        full_spiral_inner = gdspy.boolean(spiral_inner_polys, None, 'or')
        full_spiral_outer = gdspy.boolean(spiral_outer_polys, None, 'or')

        # 3. Perform Subtraction
        cutters_inner = gdspy.boolean([cElectrodeTop, cElectrodeBot], None, 'or')
        final_spiral_inner = gdspy.boolean(full_spiral_inner, cutters_inner, 'not', layer=1)

        cutters_outer = gdspy.boolean([cElectrodeTopo, cElectrodeBoto], None, 'or')
        final_spiral_outer = gdspy.boolean(full_spiral_outer, cutters_outer, 'not', layer=1)

        # 4. Add Result to Cell
        if final_spiral_inner is not None:
            main_cell.add(final_spiral_inner)
        if final_spiral_outer is not None:
            main_cell.add(final_spiral_outer)

        # 5. Add Jumpers
        jxy1 = np.vstack((jxylow, jxyup))
        jxy2 = np.vstack((jxyblow, jxybup))
        jElectrodeTop = gdspy.Polygon(jxy1, layer=1)
        jElectrodeBot = gdspy.Polygon(jxy2, layer=1)

        main_cell.add(jElectrodeTop)
        main_cell.add(jElectrodeBot)

# ---------------------------------------------------------
        # Part 6: Electrodes (Calculated Individually) - FIXED BRIDGES
        # ---------------------------------------------------------
        if final_jxyup is not None and final_jxybup is not None:

            ethick = 500
            gapAngle = np.pi/24

            # 1. Base Radius (Same for both sides)
            e1_rad = np.sqrt(final_jxyup[0,0]**2 + final_jxyup[0,1]**2) * 1.1

            # Growth/Flare settings
            r_start = e1_rad + ethick
            r_end = r_start + 1000
            r_growth = np.linspace(r_start, r_end, 100)

            # --------------------
            # RIGHT SIDE ELECTRODE
            # --------------------
            # Angles
            topAngle = iniangj[0] + DegreeAngle
            start_angle_rad = topAngle - 5 * (np.pi/180)
            start_angle_rad_out = topAngle + 5 * (np.pi/180)
            end_angle_rad = iniangj[0] + np.pi/2 - gapAngle

            # Arc Generation
            t_valsin = np.linspace(start_angle_rad, end_angle_rad, 100)
            t_valsout = np.linspace(start_angle_rad_out, end_angle_rad, 100)

            rx_in = e1_rad * np.cos(t_valsin)
            ry_in = e1_rad * np.sin(t_valsin)
            rx_out = r_growth * np.cos(t_valsout)
            ry_out = r_growth * np.sin(t_valsout)

            # Flat Top Clipping
            y_max = (e1_rad + ethick) * np.sin(end_angle_rad)
            ry_in = np.minimum(ry_in, y_max)
            ry_out = np.minimum(ry_out, y_max)

            # Polygon
            poly_right_pts = np.vstack((
                np.column_stack((rx_in, ry_in)),
                np.column_stack((np.flip(rx_out), np.flip(ry_out)))
            ))
            main_cell.add(gdspy.Polygon(poly_right_pts, layer=1))

            # Bridge (Right) - FIXED ORDER
            # Connects: Spiral Top -> Elec Top -> Elec Bottom -> Spiral Bottom
            p_spiral_in = final_jxyup[0]   # Top of Spiral Tip
            p_spiral_out = final_jxyup[1]  # Bottom of Spiral Tip
            p_elec_in = np.array([rx_in[0], ry_in[0]])    # Bottom of Elec Start (Inner Rad)
            p_elec_out = np.array([rx_out[0], ry_out[0]]) # Top of Elec Start (Outer Rad)

            # Correct Perimeter Order
            bridge_right = np.array([p_spiral_in, p_elec_out, p_elec_in, p_spiral_out])
            main_cell.add(gdspy.Polygon(bridge_right, layer=1))

            # --------------------
            # LEFT SIDE ELECTRODE
            # --------------------
            # Calculate angles by reflecting across Y-axis (Pi - angle)
            start_angle_rad_L = np.pi - start_angle_rad
            start_angle_rad_out_L = np.pi - start_angle_rad_out
            end_angle_rad_L = np.pi - end_angle_rad

            # Arc Generation (Left)
            # Note: We iterate from "bottom" (near spiral) to "top" (flat edge)
            t_valsin_L = np.linspace(start_angle_rad_L, end_angle_rad_L, 100)
            t_valsout_L = np.linspace(start_angle_rad_out_L, end_angle_rad_L, 100)

            rx_in_L = e1_rad * np.cos(t_valsin_L)
            ry_in_L = e1_rad * np.sin(t_valsin_L)
            rx_out_L = r_growth * np.cos(t_valsout_L)
            ry_out_L = r_growth * np.sin(t_valsout_L)

            # Flat Top Clipping (Same y_max)
            ry_in_L = np.minimum(ry_in_L, y_max)
            ry_out_L = np.minimum(ry_out_L, y_max)

            # Polygon
            poly_left_pts = np.vstack((
                np.column_stack((rx_in_L, ry_in_L)),
                np.column_stack((np.flip(rx_out_L), np.flip(ry_out_L)))
            ))
            main_cell.add(gdspy.Polygon(poly_left_pts, layer=1))

            # Bridge (Left) - FIXED ORDER
            # Connects: Spiral Top (Left) -> Elec Top (Left) -> Elec Bottom (Left) -> Spiral Bottom (Left)
            p_spiral_in_L = final_jxybup[0] # Inner tip of left spiral (Top)
            p_spiral_out_L = final_jxybup[1] # Outer tip of left spiral (Bottom)
            p_elec_in_L = np.array([rx_in_L[0], ry_in_L[0]])   # Bottom of Elec (Inner Rad)
            p_elec_out_L = np.array([rx_out_L[0], ry_out_L[0]]) # Top of Elec (Outer Rad)

            # Correct Perimeter Order
            bridge_left = np.array([p_spiral_in_L, p_spiral_out_L, p_elec_out_L, p_elec_in_L])
            main_cell.add(gdspy.Polygon(bridge_left, layer=1))

            # Filename Text
            txt_pos = (-1 * rx_out[-1], -1 * np.sqrt(rx_out[-1]**2 + ry_out[-1]**2))
            main_cell.add(gdspy.Text(strv, 100, txt_pos, layer=1))

        # Center Mark
        r0 = 0.88 * (cWater / freq[0]) * 1e6
        mk_ang = np.linspace(0, 2*np.pi, 100)
        path_mk = np.column_stack((r0 * np.cos(mk_ang), r0 * np.sin(mk_ang)))
        main_cell.add(gdspy.FlexPath(path_mk, 10, layer=2))

        # Save
        libname = f"{strv}.gds"
        print(f"Saving {libname}...")
        gdspy.write_gds(libname, cells=[main_cell])

generate_designs()
print("Done! Rake design generated with calculated left/right electrodes.")

Processing Rake design s=6 with fixed electrodes...
Saving RakeTrap-16-(44)-Cont12.gds...
Processing Rake design s=8 with fixed electrodes...
Saving RakeTrap-18-(44)-Cont12.gds...
Done! Rake design generated with calculated left/right electrodes.


In [ ]:
!rm *gds